In [5]:
import pandas as pd
import numpy as np
import gc, sqlite3
from tratamento_de_arquivos_e_dados.tratador_de_arquivos import tratador_de_arquivos

In [13]:
# bos ou rat
tipo = 'rat'

In [14]:
df_dados_para_classificar = tratador_de_arquivos().registros_para_classificar(rat_ou_bos=tipo)

In [15]:
df_dados_para_classificar

,RAT.NUM_ATIVIDADE,MUNICIPIO,LOGRADOURO,DES_ENDERECO,COMPLEMENTO_ENDERECO,NOME_BAIRRO,LOGRADOURO2,DES_ENDERECO2,SETOR,CIA


In [9]:
df_classificadores = tratador_de_arquivos().get_df_classif()
df_dados_para_classificar.loc[:,'SETOR'] = df_dados_para_classificar.apply(
    lambda row: tratador_de_arquivos().classifica_setor(row, df_classificadores), axis=1
)
tratador_de_arquivos().classifica_cia(df_dados_para_classificar)

In [11]:
new_classif_dict = [
    {
        'nrat': dados_para_classificar_em_lista[0],
        'setor': dados_para_classificar_em_lista[-2],
        'cia': dados_para_classificar_em_lista[-1]
    }
    for dados_para_classificar_em_lista in df_dados_para_classificar.values
    if (
        dados_para_classificar_em_lista[-1] != 'CIA_INDEFINIDA' or
        dados_para_classificar_em_lista[-2] != 'SETOR_INDEFINIDO'
    )
]

new_classif_dict

[{'nrat': '2021-002320137-001', 'setor': 'NITEROI', 'cia': '142 CIA'},
 {'nrat': '2021-005295603-001', 'setor': 'SAO JOSE', 'cia': '139 CIA'},
 {'nrat': '2021-008040000-001', 'setor': 'PLANALTO', 'cia': '139 CIA'},
 {'nrat': '2021-008952827-001', 'setor': 'HIPER CENTRO', 'cia': '53 CIA'}]

In [12]:
query_update = '''
UPDATE tbl_{}_geral
    SET "SETOR" = ?,
    "CIA" == ?
WHERE
    "RAT.NUM_ATIVIDADE" = ?
'''.format(tipo)

with sqlite3.connect('gdo.db') as conn:
    cursor = conn.cursor()
    for dict_class in new_classif_dict:
        cursor.execute(query_update, (dict_class['setor'], dict_class['cia'], dict_class['nrat']))
        conn.commit()
conn.close()